In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from datetime import datetime

In [2]:
#每个页面抓取评论数方式相同,应建立评论数抽取函数
#找到返回了评论数的url模板,对比发现标识文章的是&newsid,相同部分为comos-,其他使用{}参数替换
#这里的channel也是参数,不过暂时爬取gn(国内新闻评论)
commentURL='http://comment5.news.sina.com.cn/page/info?version=1&format=js\
&channel=gn&newsid=comos-{0}&group=0&compress=1&ie=utf-8&oe=utf-8&page=1\
&show_reply=1&page_size=20'

def getCommentCount(newsurl):
    #先从newsurl内取得newsid
    m=re.search('doc-i(.+).shtml',newsurl)
    newsid=m.group(1)
    #print(newsid)
    #将newsid放到请求模板中,发送请求
    resinfo=requests.get(commentURL.format(newsid))
    resinfo.encoding='utf-8'
    #将得到的json转化为字典
    jd = json.loads(resinfo.text.strip('var data='))
    #得到内嵌字典里的评论数
    return jd['result']['count']['total']

In [3]:
#现在可以直接使用函数爬取该站点channel=gn的评论数了
#如果是科技类就要另外做判断
news='http://tech.sina.com.cn/csj/2018-01-17/doc-ifyqqieu7096578.shtml'
newsurl='http://news.sina.com.cn/china/xlxw/2018-01-17/doc-ifyqrewi9270282.shtml'
print(getCommentCount(newsurl))
#print(getCommentCount(news)) #这个不是国内新闻就请求不到评论数

3922


In [4]:
#抓取新闻函数
def getNewsDetail(newsurl):
    result={}
    res = requests.get(newsurl)
    res.encoding='utf-8'
    soup=BeautifulSoup(res.text,'html.parser')
    result['title']=soup.select('.main-title')[0].text
    dt=datetime.strptime(soup.select('.date-source')[0].contents[1].text,'%Y年%m月%d日 %H:%M')
    result['time']=datetime.strftime(dt,'%Y-%m-%d')
    result['newssource']=soup.select('.date-source a')[0].text
#     result['article']=[]
#     for p in soup.select('.article p')[:-1]:
#         result['article'].append(p.text.strip())
#     result['article']='\n\n'.join( result['article'])
    result['article']='\n\n'.join([p.text.strip() for p in soup.select('.article p')[:-1]])
    result['editor']=soup.select('.show_author')[0].text.strip('责任编辑：').strip()
    result['comments']=getCommentCount(newsurl)
    return result

In [5]:
newsurl='http://news.sina.com.cn/china/xlxw/2018-01-17/doc-ifyqrewi9270282.shtml'
Detail=getNewsDetail(newsurl)
print (Detail)

{'title': '习近平亲民外交六个瞬间', 'time': '2018-01-17', 'newssource': '央视新闻', 'article': '原标题：习近平亲民外交六个瞬间\n\n作为一国元首，常在镜头前出现的习近平很少有机会展现他个人性情的一面。但当他与多年未谋面的老友重聚时，或在繁忙的外事活动间隙里，人们看到了一个不一样的习近平。\n\nAs the busy leader of a country of over 1.3 billion people， Chinese President Xi Jinping rarely has a chance to show his personal side to the public。 But when revisiting old friends or taking a break from diplomatic tasks， he has revealed glimpses of his personality to the world。\n\n三十年的友谊，你看看\n\nIn 1985， the Dvorchaks hosted Xi during his visit to Iowa。 Xi invited them to China 30 years later\xa0and hosted them at the Diaoyutai State Guesthouse。\n\n我过去，您别站起来了\n\nXi awarded medals to Russian veterans for their contributions to China’s victory against Japanese aggression and the World Anti-Fascist War。 Many of the veterans attending the event had limited mobility， so Xi brought the medals over to them。\n\n我也是了却一个心愿\n\nIn 2001， Jim Bacon， then premier of Tasmania， met Xi in China and invited him to visit。 In 

In [6]:
#为了批量爬取文章,需要找到文章列表及对应url,一般都会分页,列表信息可能在js/hxr/doc下
import requests
import json
#新浪微博列表url
listurl='http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&\
cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&\
show_num=22&tag=1&format=json&page=3&callback=newsloadercallback&_=1516260698808'
res=requests.get(listurl)
#json.loads只能加载字典,要把一些多出来的字串截掉
jd=json.loads(res.text.lstrip('  newsloadercallback(').rstrip(');'))
jd['result']['data']

[{'column': 'mtjj',
  'comment_channel': 'gn',
  'createtime': '1516319787',
  'ext1': '',
  'ext2': 'gn:comos-fyquixe4294264:0',
  'ext3': '',
  'ext4': 'gn:comos-fyquixe4294264:0',
  'ext5': '原标题：“解禁”第一天： 你和飞机上打手机还隔着一个行业利益\n本报记者 付丽丽',
  'id': '1-1-35197575',
  'img': '',
  'keywords': '民航,科技日报,惯性导航',
  'level': '0',
  'media_name': '科技日报',
  'media_type': '',
  'old_level': '2',
  'title': '飞机上打电话有多远？科技日报:隔着一个行业利益',
  'url': 'http://news.sina.com.cn/c/sd/2018-01-19/doc-ifyquixe4294264.shtml'},
 {'column': 'gdxw1',
  'comment_channel': 'gn',
  'createtime': '1516319293',
  'ext1': '',
  'ext2': 'gn:comos-fyquptv7842380:0',
  'ext3': '',
  'ext4': 'gn:comos-fyquptv7842380:0',
  'ext5': '原标题：媒体：东北并非无药可救不能因少数人就否定东北\n据报道，京东集团创始人刘强东近日密集拜会东北三省省委书记、省长，与三省签署战略合作协议，承诺未来三年在东北投资超过200亿元。他表示，最近几年大家似乎都不看好东北，不可否认东北三省的经济确实遇到不小的困难，但是不要忘了，…',
  'id': '1-1-35197325',
  'img': '',
  'keywords': '东北,营商环境,转型',
  'level': '1',
  'media_name': '人民网',
  'media_type': '',
  'old_level': '1',
  'title': '北青报：东北

In [7]:
#循环获得所有文章的url
urllists=[ newsurl['url'] for newsurl in jd['result']['data']]
len(urllists)
urllists

['http://news.sina.com.cn/c/sd/2018-01-19/doc-ifyquixe4294264.shtml',
 'http://news.sina.com.cn/c/nd/2018-01-19/doc-ifyquptv7842380.shtml',
 'http://news.sina.com.cn/w/2018-01-19/doc-ifyquixe4290062.shtml',
 'http://news.sina.com.cn/o/2018-01-19/doc-ifyquixe4289514.shtml',
 'http://news.sina.com.cn/c/nd/2018-01-19/doc-ifyquptv7841558.shtml',
 'http://news.sina.com.cn/c/nd/2018-01-19/doc-ifyquixe4284178.shtml',
 'http://news.sina.com.cn/c/nd/2018-01-19/doc-ifyquixe4282654.shtml',
 'http://news.sina.com.cn/c/nd/2018-01-19/doc-ifyquptv7839279.shtml',
 'http://news.sina.com.cn/c/zs/2018-01-19/doc-ifyquixe4278186.shtml',
 'http://news.sina.com.cn/c/2018-01-19/doc-ifyqtycw9958684.shtml',
 'http://news.sina.com.cn/o/2018-01-19/doc-ifyquixe4277144.shtml',
 'http://news.sina.com.cn/o/2018-01-19/doc-ifyquixe4268821.shtml',
 'http://news.sina.com.cn/c/2018-01-19/doc-ifyquixe4265479.shtml',
 'http://news.sina.com.cn/c/2018-01-19/doc-ifyqtycw9939005.shtml',
 'http://news.sina.com.cn/c/2018-01-19/do

In [8]:
#为了同时获取多个分页的url列表,比较修改分页后listurl的参数改变
# 'http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&\
# cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&\
# show_num=22&tag=1&format=json&page=3&callback=newsloadercallback&_=1516260698808'

# 'http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&\
# cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&\
# show_num=22&tag=1&format=json&page=2&callback=newsloadercallback&_=1516262493165'
#发现是使用page来控制分页
import requests
import json

def getDetails(count):
    url='http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&\
cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&\
show_num=22&tag=1&format=json&page={}&callback=newsloadercallback&_=1516262493165'
    #for循环将多个页面的链接列表加到一起
    details=[]
    for n in range(1,count+1):
        res=requests.get(url.format(n))
        jd=json.loads(res.text.lstrip('  newsloadercallback(').rstrip(');'))
        #取得所有文章详情
        details.extend([getNewsDetail(newsurl['url']) for newsurl in jd['result']['data']])    
    return details


In [13]:
#指定获取多少页的文章详情,放在列表中,每个列表索引指向一个字典存放一篇文章详情
news=getDetails(2)
print('total:',len(news))
print(news)

total: 44
[{'title': '男子报复邻居国外买枪获刑被遣返 国内也能追诉？', 'time': '2018-01-19', 'newssource': '澎湃新闻', 'article': '原标题：法治课|四川男子国外买枪获刑被遣返，国内检方可否追诉\n\n澎湃新闻记者 林平\n\n因受不了邻居噪音欲报复，四川一男子跑到巴基斯坦买枪弹被捕获刑，在服刑期内遣返归国时，其又被国内检察机关以“非法持有枪支”罪名重新批捕。\n\n中国公民在国外作案获刑后，归国时还将面临何种司法境遇？国内司法机关是否有权进行追诉和审判？有专家分析指出，这一问题涉及刑事管辖权原则，尽管涉案男子已在国外被判处刑罚，但并不影响我国依据国内刑法对其进行追诉，“可据其实际服刑情况，在具体量刑上给予适当从宽。”\n\n据封面新闻报道，2016年，居住在四川省崇州市三江镇的王姓男子因住所隔壁加工厂噪音污染问题多次与邻居发生冲突。多次沟通无果后，他竟远赴巴基斯坦购买了4支手枪和200发子弹欲实施报复。\n\n在回国途中，王某打消了报复念头，遂在中巴边境一山谷处将枪弹发射，其间被闻声赶到的巴基斯坦警方抓获。\n\n2016年6月23日，王某被当地法院分别判处4年和5年监禁，并驱逐出境。2017年7月31日，巴基斯坦内政部在新疆红其拉普口岸将王某移交中方处理。\n\n最终，崇州检察院以非法持有枪支罪批捕王某。\n\n这一批捕决定源自何据？国内司法机关为何不直接执行巴基斯坦的刑事判决？\n\n“此案涉及到刑事管辖权的问题。”北京师范大学中国刑法研究所副所长彭新林表示，因刑事判决涉及国家司法主权，中国是不可能直接执行巴基斯坦的刑事判决，而且对于域外判决我国采取的是消极承认态度，也就是说，虽然历经国外审判，但是国内还是可以以“属人管辖”的原则进行追诉。\n\n刑事管辖原则是指刑法对地和对人的效力，也就是解决一个国家的刑事管辖权的问题。\n\n澎湃新闻注意到，一般而言，刑事管辖权具有四个原则：第一属地管辖；第二属人管辖；第三保护管辖；第四普遍管辖。\n\n其中，属地管辖原则被誉为是“黄金原则”。彭新林分析说，涉案男子被巴基斯坦警方控制，当地司法机关对其进行追诉和判刑，正是基于属地管辖权，“当有多个国家或地区对同一起案件拥有管辖权的情况下，一般以属地管辖为优先。”\n\n此外，在案情上，涉案男子非法持有

In [15]:
#使用资料格式datafram
import pandas
df =pandas.DataFrame(news)
df.head(10)

,article,comments,editor,newssource,time,title
0,原标题：法治课|四川男子国外买枪获刑被遣返，国内检方可否追诉\n\n澎湃新闻记者 林平\n\...,0,张玉,澎湃新闻,2018-01-19,男子报复邻居国外买枪获刑被遣返 国内也能追诉？
1,原标题：湖北省第十三届人民代表大会代表名单公布：共726名\n\n武汉市（125名）\n\n...,0,张玉,澎湃新闻,2018-01-19,湖北第十三届人民代表大会代表名单公布：共726名
2,原标题：伊朗感谢中方对桑吉号搜救 中方：将安排机器人搜索\n\n中新网1月19日电 据中国驻...,0,初晓慧,中国新闻网,2018-01-19,伊朗感谢中国搜救桑吉轮 中方：将安排机器人搜索
3,原标题：落马贪官自述腐败心理：不甘心“进肚肥肉”再吐出\n\n中新网1月19日电 安徽省纪委...,0,初晓慧,中国新闻网,2018-01-19,落马贪官自述腐败心理:不甘心“进肚肥肉”再吐出
4,原标题：教育部：自学考试不会取消 法律法规无调整变化\n\n中新网1月19日电 教育部网站1...,0,初晓慧,中国新闻网,2018-01-19,教育部：自学考试不会取消 法律法规无调整变化
5,原标题：江西厅级干部调整：欧阳海泉任省民政厅正厅级副厅长\n\n信息日报江西政读获悉，江西任...,0,张玉,澎湃新闻,2018-01-19,江西厅级干部调整:欧阳海泉任民政厅正厅级副厅长
6,原标题：蒋经国之孙蒋万安：“我不会参加2018年台北市长的选举”\n\n岛内2018县市首长...,0,张玉,澎湃新闻,2018-01-19,蒋经国之孙蒋万安：今年我不会参加台北市长选举
7,原标题：台媒：北京薪资超越台北，对台有弊有利\n\n台湾《工商时报》19日社论指出，在最近台...,0,张义凌,中国新闻网,2018-01-19,北京薪资超越台北 台媒建议蔡当局开放陆资入台
8,原标题：印媒炒作中国在瓜达尔港建“核潜艇基地” 巴基斯坦否认\n\n参考消息网1月19日报道...,1,初晓慧,参考消息,2018-01-19,印媒炒作中国在瓜达尔港建核潜艇基地 巴方否认
9,原标题：蔡英文年终奖71万新台币 被讽：全民共贫唯她独乐\n\n海外网1月19日电 临近春节...,6,初晓慧,人民日报海外版-海外网,2018-01-19,蔡英文年终奖15万人民币 被讽:全民共贫唯她独乐


In [18]:
#保存至Excel
#pip3 install openpyxl
df.to_excel('新浪新闻.xlsx')

In [20]:
#保存至sqllite
import sqlite3
with sqlite3.connect('news.sqlite') as db:
    df.to_sql('new',con=db)

In [23]:
#从数据库查出
df2=pandas.read_sql_query('select * from new',con =db)
df2.head()

,index,article,comments,editor,newssource,time,title
0,0,原标题：法治课|四川男子国外买枪获刑被遣返，国内检方可否追诉\n\n澎湃新闻记者 林平\n\...,0,张玉,澎湃新闻,2018-01-19,男子报复邻居国外买枪获刑被遣返 国内也能追诉？
1,1,原标题：湖北省第十三届人民代表大会代表名单公布：共726名\n\n武汉市（125名）\n\n...,0,张玉,澎湃新闻,2018-01-19,湖北第十三届人民代表大会代表名单公布：共726名
2,2,原标题：伊朗感谢中方对桑吉号搜救 中方：将安排机器人搜索\n\n中新网1月19日电 据中国驻...,0,初晓慧,中国新闻网,2018-01-19,伊朗感谢中国搜救桑吉轮 中方：将安排机器人搜索
3,3,原标题：落马贪官自述腐败心理：不甘心“进肚肥肉”再吐出\n\n中新网1月19日电 安徽省纪委...,0,初晓慧,中国新闻网,2018-01-19,落马贪官自述腐败心理:不甘心“进肚肥肉”再吐出
4,4,原标题：教育部：自学考试不会取消 法律法规无调整变化\n\n中新网1月19日电 教育部网站1...,0,初晓慧,中国新闻网,2018-01-19,教育部：自学考试不会取消 法律法规无调整变化
